<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/ifcopenshell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

print("IFCファイルを選択してください")
uploaded_ifc = files.upload()

print("Energy+.idd を選択してください")
uploaded_idd = files.upload()


IFCファイルを選択してください


Saving シミュレーション導入.ifc to シミュレーション導入.ifc
Energy+.idd を選択してください


Saving Energy+.idd to Energy+.idd


In [3]:
# -*- coding: utf-8 -*-
# IfcSpaceの実ジオメトリをEnergyPlusサーフェスへ。屋根判定を強化（上面高さ + 傾斜角）
import io, os, math
import numpy as np

import ifcopenshell
import ifcopenshell.util.element as ifc_elem
import ifcopenshell.geom as ifc_geom
from eppy.modeleditor import IDF

# ==== 入力 ====
EPLUS_IDD = "/content/Energy+.idd"
IFC_PATH  = "/content/input.ifc"
IDF_OUT   = "/content/out.idf"

assert os.path.isfile(EPLUS_IDD)
assert os.path.isfile(IFC_PATH)

# ==== IDF初期化（空IDFは不可：最小IDFで）====
IDF.setiddname(EPLUS_IDD)
idf = IDF(io.StringIO("Version,25.1;"))  # ← IDDに合わせて必要なら数字を変更

# ==== ヘッダ ====
idf.newidfobject("SITE:LOCATION", Name="Tokyo", Latitude=35.68, Longitude=139.76, Time_Zone=9, Elevation=10.0)
idf.newidfobject("TIMESTEP", Number_of_Timesteps_per_Hour=6)
idf.newidfobject("BUILDING", Name="IFC_Import_Geom", North_Axis=0.0, Terrain="City",
                 Loads_Convergence_Tolerance_Value=0.04, Temperature_Convergence_Tolerance_Value=0.4)

# ==== IFC ====
ifc = ifcopenshell.open(IFC_PATH)

# Pset互換ラッパ
def safe_get_psets(elem):
    try:
        return ifc_elem.get_psets(elem)
    except Exception:
        try:
            import ifcopenshell.util.pset as ifc_pset
            return ifc_pset.get_psets(elem)
        except Exception:
            return {}

# 代表U値（見つからなければ既定）
def get_ifc_u(e):
    def pick(ps):
        for v in (ps or {}).values():
            if isinstance(v, dict) and "ThermalTransmittance" in v and v["ThermalTransmittance"]:
                try:
                    u = float(v["ThermalTransmittance"])
                    if u > 0: return u
                except: pass
        return None
    u = pick(safe_get_psets(e))
    if u: return u
    try:
        t = ifc_elem.get_type(e)
        if t: return pick(safe_get_psets(t))
    except: pass
    return None

def representative_u(names, default):
    vals = []
    for n in names:
        for e in ifc.by_type(n):
            u = get_ifc_u(e)
            if u and u > 0: vals.append(u)
    if vals:
        vals.sort()
        return vals[len(vals)//2]
    return default

DEFAULTS = dict(Wall=1.5, Roof=0.8, Floor=1.2, Window=3.0)
U_wall  = representative_u(["IfcWall","IfcWallStandardCase"], DEFAULTS["Wall"])
U_roof  = representative_u(["IfcRoof"], DEFAULTS["Roof"])
U_floor = representative_u(["IfcSlab"], DEFAULTS["Floor"])

def ensure_nomass_construction(cat, u):
    R = 1.0/max(u,1e-6)
    mat = idf.newidfobject("MATERIAL:NOMASS",
                           Name=f"{cat}_NoMass_R{R:.3f}",
                           Roughness="MediumRough", Thermal_Resistance=R,
                           Thermal_Absorptance=0.9, Solar_Absorptance=0.7, Visible_Absorptance=0.7)
    return idf.newidfobject("CONSTRUCTION", Name=f"{cat}_U{u:.3f}", Outside_Layer=mat.Name)

CON_WALL  = ensure_nomass_construction("Wall",  U_wall)
CON_ROOF  = ensure_nomass_construction("Roof",  U_roof)
CON_FLOOR = ensure_nomass_construction("Floor", U_floor)

# ==== ifcopenshell.geom 設定（存在するキーだけ安全にセット）====
settings = ifc_geom.settings()
def set_opt(s, key, val):
    try:
        s.set(getattr(s, key), val)
    except Exception:
        pass
set_opt(settings, "USE_WORLD_COORDS", True)
set_opt(settings, "INCLUDE_CURVES", False)
set_opt(settings, "DISABLE_TRIANGULATION", False)

def tri_normal(p0, p1, p2):
    v1 = p1 - p0; v2 = p2 - p0
    n = np.cross(v1, v2); norm = np.linalg.norm(n)
    return n/norm if norm else np.array([0.0,0.0,0.0])

# ==== しきい値（調整可）====
Z_ROOF_TILT = 0.3     # |n.z|>=0.3 を屋根/床候補にする（≈水平から60°以内も可）
Z_FLOOR_TILT = -0.3
TOP_EPS = 0.20        # そのスペースの「上面高さ」から±0.2m以内なら屋根扱いを優先
BOT_EPS = 0.20        # 下面高さ ±0.2m は床優先

spaces = ifc.by_type("IfcSpace")
print("Total IfcSpace:", len(spaces))
surf_count = 0

for i, sp in enumerate(spaces, start=1):
    zone_name = sp.Name or f"Space_{i}"
    idf.newidfobject("ZONE", Name=zone_name)

    try:
        shp = ifc_geom.create_shape(settings, sp)
    except Exception as e:
        print(f"[WARN] shape failed for {zone_name}: {e}")
        continue

    verts = np.array(shp.geometry.verts, dtype=float).reshape((-1, 3))
    faces = np.array(shp.geometry.faces, dtype=int).reshape((-1, 3))

    # スペースの上下端を把握（屋根/床の優先判定に使う）
    z_top = np.quantile(verts[:,2], 0.95)   # 外れ値に鈍感
    z_bot = np.quantile(verts[:,2], 0.05)

    for fi, (a,b,c) in enumerate(faces, start=1):
        p0, p1, p2 = verts[a], verts[b], verts[c]
        n = tri_normal(p0, p1, p2)
        zmean = (p0[2]+p1[2]+p2[2])/3.0

        # まず「高さで優先判定」→ 次に「法線しきい値」で補強
        if zmean >= z_top - TOP_EPS and n[2] > 0:     # 上端近傍 & 上向き
            stype, con, bc = "Roof", CON_ROOF.Name, "Outdoors"
        elif zmean <= z_bot + BOT_EPS and n[2] < 0:   # 下端近傍 & 下向き
            stype, con, bc = "Floor", CON_FLOOR.Name, "Ground"
        else:
            if   n[2] >=  Z_ROOF_TILT: stype, con, bc = "Roof",  CON_ROOF.Name,  "Outdoors"
            elif n[2] <=  Z_FLOOR_TILT:stype, con, bc = "Floor", CON_FLOOR.Name, "Ground"
            else:                       stype, con, bc = "Wall",  CON_WALL.Name,  "Outdoors"

        idf.newidfobject(
            "BUILDINGSURFACE:DETAILED",
            Name=f"{i}:{stype}:{fi}",
            Surface_Type=stype,
            Construction_Name=con,
            Zone_Name=zone_name,
            Outside_Boundary_Condition=bc,
            Sun_Exposure="SunExposed",
            Wind_Exposure="WindExposed",
            View_Factor_to_Ground="autocalculate",
            Number_of_Vertices=3,
            Vertex_1_Xcoordinate=float(p0[0]), Vertex_1_Ycoordinate=float(p0[1]), Vertex_1_Zcoordinate=float(p0[2]),
            Vertex_2_Xcoordinate=float(p1[0]), Vertex_2_Ycoordinate=float(p1[1]), Vertex_2_Zcoordinate=float(p1[2]),
            Vertex_3_Xcoordinate=float(p2[0]), Vertex_3_Ycoordinate=float(p2[1]), Vertex_3_Zcoordinate=float(p2[2]),
        )
        surf_count += 1

print("Generated triangle surfaces:", surf_count)
idf.saveas(IDF_OUT)
print("Saved:", IDF_OUT)


Total IfcSpace: 14
Generated triangle surfaces: 486
Saved: /content/out.idf
